In [ ]:
import great_expectations as gx
import os

In [ ]:
context = gx.get_context(
    ge_cloud_organization_id="728c7fee-386f-417a-b313-a7e73595c994",
    cloud_mode=True
)

In [ ]:
asset = context.get_datasource("cockroachdb").get_asset("ao_info_carpex_super_region")

In [ ]:
# datasource = context.sources.add_sql(
#     "cockroachdb",
#     create_temp_table=False,
#     connection_string=os.environ["COCKROACH_CONNECTION_STRING"]
# )

# for name in ["ao_info_carpex", "ao_info_greenlevel", "ao_info_carpex_super_region", "ao_info_peakcity"]:
#     datasource.add_table_asset(f"{name}", f"{name}")
# asset = datasource.add_table_asset("backblast", "backblast").add_splitter_year_and_month_and_day("date")

In [ ]:
try:
    context.add_expectation_suite("backblast")
except (ValueError, gx.exceptions.GreatExpectationsError):
    pass

In [ ]:
validator = context.get_validator(batch_request=asset.build_batch_request(), expectation_suite_name="ao_info")

In [ ]:
validator.expect_column_values_to_not_be_null("ao_day_of_week_int", mostly=0.40)

In [ ]:
validator.save_expectation_suite()

In [ ]:
validator.expect_table_row_count_to_be_between(5, 130, meta={"notes": {"content": "A region should only have a handful of AOs"}})

In [ ]:
validator.expect_table_columns_to_match_ordered_list(validator.columns())

In [ ]:
validator.expect_column_values_to_be_unique("ao")

In [ ]:
for column in validator.columns():
    validator.expect_column_values_to_not_be_null(column)

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)

In [ ]:
validator.expect_column_values_to_be_between("ao_lon", -79.5, -78)

In [ ]:
validator.expect_column_values_to_be_between("ao_lat", 35, 36)

In [ ]:
res = context.assistants.onboarding.run(batch_request=asset.build_batch_request())

In [ ]:
validator.expect_column_distinct_values_to_be_in_set("ao_type", ["1stf", "2ndf", "3rdf"])

In [ ]:
validator.expect_column_values_to_be_between("ao_day_of_week_int", 1, 6)

In [ ]:
checkpoint_name = "ao_info"  # name your checkpoint here!

checkpoint_config = {
  "name": checkpoint_name,
  "validations": [{
      "expectation_suite_name": "ao_info",
      "expectation_suite_ge_cloud_id": context.get_expectation_suite("ao_info").ge_cloud_id,
      "batch_request": {
          "datasource_name": "cockroachdb",
          "data_asset_name": "ao_info_carpex",
      },
  },
  {
      "expectation_suite_name": "ao_info",
      "expectation_suite_ge_cloud_id": context.get_expectation_suite("ao_info").ge_cloud_id,
      "batch_request": {
          "datasource_name": "cockroachdb",
          "data_asset_name": "ao_info_carpex_super_region",
      },
  },
  {
      "expectation_suite_name": "ao_info",
      "expectation_suite_ge_cloud_id": context.get_expectation_suite("ao_info").ge_cloud_id,
      "batch_request": {
          "datasource_name": "cockroachdb",
          "data_asset_name": "ao_info_churham",
      },
  },
    {
      "expectation_suite_name": "ao_info",
      "expectation_suite_ge_cloud_id": context.get_expectation_suite("ao_info").ge_cloud_id,
      "batch_request": {
          "datasource_name": "cockroachdb",
          "data_asset_name": "ao_info_peakcity",
      },
  }],
}

checkpoint = context.add_or_update_checkpoint(**checkpoint_config)

print(checkpoint)

# Optionally run the Checkpoint:
# result = checkpoint.run()


In [ ]:
asset = context.get_datasource("cockroachdb").get_asset("backblast")

In [ ]:
res = context.assistants.onboarding.run(batch_request=asset.build_batch_request(), include_column_names=["date", "store_date"])

In [ ]:
validator.save_expectation_suite()